In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqFeature

In [13]:
dup_aa = 'TERDTSTSQSTVLDTTTSKH'
dup_seq = 'ACAGAAAGAGACACCAGCACCTCACAATCCACTGTGCTCGACACAACCACATCAAAACAC'

In [ ]:
#RSVB AY333364 is the "prototype" BA sequence, with a perfect duplication
#Use this as the G gene

In [106]:
for record in SeqIO.parse(open('rsv_b_AY333364.gb', 'r'), "gb"):
    ba_g_seq_extra = record.seq
    for feature in record.features:
        if feature.type =='CDS':
            ba_g_translation = feature.qualifiers['translation'][0]
            ba_g_seq = feature.location.extract(ba_g_seq_extra)
            loc_template = ba_g_seq.find(dup_seq)
            loc_dup = loc_template+60
            ba_g_start = feature.location.start


In [116]:
print(ba_g_seq)

ATGTCCAAAAACAAGAATCAACGCACTGCCAGGACTCTAGAAAAGACCTGGGATACTCTTAATCATCTAATTGTAATATCCTCTTGTTTATACAAATTAAATTTAAAATCTATAGCACAAATAGCACTATCAGTTTTGGCAATGATAATCTCAACCTCTCTCATAATTGCAGCCATAATATTCATCATCTCTGCCAATCACAAAGTTACACTAACAACTGTCACAGTTCAAACAATAAAAAACCACACTGAGAAAAACATCACCACTTACCTTACTCAAGTCTCACCAGAAAGGGTTAGCCCATCCAAACAACTCACAACCACACCACCAATCTACACAAACTCAGCCACAATATCACCTAATACAAAATCAGAAACACACCATACAACAGCACAAACCAAAGGCAGAACCACCACTCCAACACAGAACAACAAGCCAAGCACAAAACCACGTCCAAAAAATCCACCAAAAAAACCAAAAGATGATTACCATTTTGAAGTGTTCAACTTCGTTCCCTGTAGTATATGTGGCAACAATCAACTCTGCAAATCCATTTGCAAAACAATACCAAGCAATAAACCAAAGAAAAAACCAACCATAAAACCCACAAACAAACCACCCACCAAAACCACAAACAAAAGAGACCCAAAAAAACTAGCCAAAACACTGAAAAAAGAAACCACCATCAACCCAACAAAAAAACCAACCCCCAAGACCACAGAAAGAGACACCAGCACCTCACAATCCACTGTGCTCGACACAACCACATCAAAACACACAGAAAGAGACACCAGCACCTCACAATCCACTGTGCTCGACACAACCACATCAAAACACACAATCCAACAGCAATCCCTCCACTCAACCACCCCCGAAAACACACCCAACTCCACACAAACACCCACAGCATCCGAGCCCTCCACATCAAATTCCACCCAAAAACTCTAG


In [119]:
print(str(ba_g_seq_extra[ba_g_start:ba_g_start+len(old_g_seq)+60].translate()))

MSKNKNQRTARTLEKTWDTLNHLIVISSCLYKLNLKSIAQIALSVLAMIISTSLIIAAIIFIISANHKVTLTTVTVQTIKNHTEKNITTYLTQVSPERVSPSKQLTTTPPIYTNSATISPNTKSETHHTTAQTKGRTTTPTQNNKPSTKPRPKNPPKKPKDDYHFEVFNFVPCSICGNNQLCKSICKTIPSNKPKKKPTIKPTNKPPTKTTNKRDPKKLAKTLKKETTINPTKKPTPKTTERDTSTSQSTVLDTTTSKHTERDTSTSQSTVLDTTTSKHTIQQQSLHSTTPENTPNSTQTPTASEPSTSNSTQKL*SYA*


In [84]:
#use AF013254 from 1999 as genome reference, but replace G gene with AY333364
#they use different stop codons, so make sure the chimera genomes makes sense:
#AF013254 uses a stop codon that is further downstream (by 4 codons)
#In AF013254 the stop codon is the TAG at the end here: CAATCACATGCTTAG, but
#In AY333364 the CAA has mutated to a TAG, making the stop codon is the first TAG here: TAGTCATATGCTTAG

#Need to insert the AY333364 G sequence into AF013254, essentially mutating CAA->TAG but keeping the downsteam sequence
#and adjust the coordinates for G and all downstream features by 60 to account for the duplication

In [123]:
#make new genome seq
new_genome_seq = ''
for record in SeqIO.parse(open('rsv_b_AF013254.gb', 'r'), "gb"):
    genome_seq = record.seq
    for feature in record.features:
        if feature.type =='CDS':
            if feature.qualifiers['gene'] == ['G']:
                g_start_old = feature.location.start
                g_end_old = feature.location.end
                old_g_seq = feature.location.extract(genome_seq)

    genome_til_G = genome_seq[:g_start_old]
    genome_after_G = genome_seq[g_end_old:]
    #the BA G sequence with the extra 4 codons to reach the stop codon of the G gene in reference genome
    ba_g_to_insert = ba_g_seq_extra[ba_g_start:ba_g_start+len(old_g_seq)+60]
    new_genome_seq = Seq(genome_til_G+ba_g_to_insert+genome_after_G)

# Create a seq record
new_record = SeqRecord(new_genome_seq,
                   id='AY333364/AF013254', # random accession number
                   name='AY333364/AF013254',
                   description='AF013254 genome with AY333364 G gene', 
                       annotations={"molecule_type": "RNA"})
    
#make new features

for record in SeqIO.parse(open('rsv_b_AF013254.gb', 'r'), "gb"):
    for feature in record.features:
        if feature.type =='source':
            feature.location = SeqFeature.FeatureLocation(SeqFeature.ExactPosition(feature.location.start.position),
                    SeqFeature.ExactPosition(feature.location.end +60), feature.location.strand)
            
        else:
            #adjust the end position of G by 60nt
            if feature.location.start==g_start_old:
                feature.location = SeqFeature.FeatureLocation(SeqFeature.ExactPosition(feature.location.start.position),
                        SeqFeature.ExactPosition(feature.location.end +60), feature.location.strand)
                if feature.type =='CDS':
                    feature.qualifiers['translation'] = feature.location.extract(new_genome_seq).translate()
            #adjust the start and end positions of all features downstream of G by 60nt
            elif feature.location.start>g_start_old:
                feature.location = SeqFeature.FeatureLocation(SeqFeature.ExactPosition(feature.location.start.position+60),
                        SeqFeature.ExactPosition(feature.location.end +60), feature.location.strand)
                if feature.type =='CDS':
                    feature.qualifiers['translation'] = feature.location.extract(new_genome_seq).translate()

        new_record.features.append(feature)

#Save as genbank
output_file = open('rsv_b_ba_reference.gb', 'w')
SeqIO.write(new_record, output_file, 'genbank')        


1

In [134]:
#check new reference genome

#old reference genome
#the new reference file should have the same gene seqs for this except for G
sequence_old_genes = {}
for record in SeqIO.parse(open('rsv_b_AF013254.gb', 'r'), "gb"):
    genome_len = len(record.seq)
    for feature in record.features:
        if feature.type == 'CDS':
            cds_seq = feature.location.extract(record.seq)
            sequence_old_genes[feature.qualifiers['gene'][0]] = cds_seq
            
#BA g gene should be from AY333364
sequence_old_genes['G']=ba_g_seq
    
for record in SeqIO.parse(open('rsv_b_ba_reference.gb', 'r'), "gb"):
    new_genome_seq = record.seq
    if len(new_genome_seq)!=genome_len+60:
        print('length mismatch')
    for feature in record.features:
        if feature.type == 'CDS':
            new_cds_seq = feature.location.extract(new_genome_seq)
            gene = feature.qualifiers['gene'][0]
            if sequence_old_genes[gene] != new_cds_seq:
                #its ok if G is different by the 12nt at the end that spans the different stop codons
                print(f'{gene} sequence mismatch')
#                 print(sequence_old_genes[gene])
#                 print(new_cds_seq)
#                 print(new_cds_seq.translate())
            

G sequence mismatch
